# Dataset generation using emobpy

In [1]:
# Initialize seed
from emobpy.tools import set_seed
set_seed()
from emobpy import Mobility, DataBase, Consumption, HeatInsulation, BEVspecs, Availability, Charging
import random
import os
import pandas as pd
import numpy as np
import re

## Init

In [2]:
config_folder='config_files'
num_users = 100  # Number of users (EVs)
simulation_hours = 8760
time_step = 0.25

In [3]:
ev_models = [
    ("Audi", "e-tron 55 quattro", 2019),
    ("Audi", "e-tron 55 quattro", 2020),
    ("Audi", "e-tron 55 quattro Premium Plus", 2019),
    ("Audi", "e-tron 55 quattro Prestige", 2019),
    ("Audi", "e-tron Sportback 50 quattro", 2020),
    ("Audi", "e-tron Sportback 55 quattro", 2020),
    
    ("BMW", "i3 22 kWh", 2014),
    ("BMW", "i3 22 kWh", 2015),
    ("BMW", "i3 22 kWh", 2016),
    ("BMW", "i3 33 kWh", 2017),
    ("BMW", "i3 33 kWh", 2018),
    ("BMW", "i3 42 kWh", 2019),
    ("BMW", "i3s 33 kWh", 2018),
    ("BMW", "i3s 42 kWh", 2019),
    ("BMW", "i3s Edition RoadStyle 42 kWh", 2020),
    
    ("Chevrolet", "Bolt EV", 2017),
    ("Chevrolet", "Bolt EV", 2018),
    ("Chevrolet", "Bolt EV", 2019),
    ("Chevrolet", "Bolt EV", 2020),
    ("Chevrolet", "Spark EV", 2014),
    ("Chevrolet", "Spark EV", 2015),
    ("Chevrolet", "Spark EV", 2016),
    
    ("FIAT", "500e", 2013),
    ("FIAT", "500e", 2016),
    ("FIAT", "500e", 2019),
    
    ("Hyundai", "IONIQ Electric 28 kWh", 2019),
    ("Hyundai", "IONIQ Electric 38.3 kWh", 2020),
    ("Hyundai", "KONA Electric 64 kWh", 2019),
    
    ("Jaguar", "I-Pace", 2019),
    ("Jaguar", "I-Pace S EV400 AWD Automatic", 2020),
    
    ("KIA", "Soul EV", 2015),
    ("KIA", "Soul EV", 2016),
    ("KIA", "Soul EV", 2017),
    ("KIA", "Soul EV", 2018),
    ("KIA", "Soul EV", 2019),
    ("KIA", "Soul EV 39 kWh", 2020),
    ("KIA", "Soul EV 64 kWh", 2020),
    ("KIA", "e-Niro 39 kWh", 2019),
    ("KIA", "e-Niro 4", 2020),
    ("KIA", "e-Niro 64 kWh", 2019),
    
    ("MINI", "Cooper SE Level I", 2020),
    ("MINI", "Cooper SE Level II", 2020),
    ("MINI", "Cooper SE Level III", 2020),
    
    ("Nissan", "Leaf S", 2013),
    ("Nissan", "Leaf S", 2014),
    ("Nissan", "Leaf S", 2015),
    ("Nissan", "Leaf S", 2016),
    ("Nissan", "Leaf S", 2017),
    ("Nissan", "Leaf S", 2018),
    ("Nissan", "Leaf S", 2019),
    ("Nissan", "Leaf S Plus", 2019),
    ("Nissan", "Leaf SL", 2011),
    ("Nissan", "Leaf SL", 2012),
    ("Nissan", "Leaf SL", 2013),
    ("Nissan", "Leaf SL", 2014),
    ("Nissan", "Leaf SL", 2015),
    ("Nissan", "Leaf SL", 2016),
    ("Nissan", "Leaf SL", 2017),
    ("Nissan", "Leaf SL", 2018),
    ("Nissan", "Leaf SL", 2019),
    ("Nissan", "Leaf SL Plus", 2019),
    ("Nissan", "Leaf SV", 2011),
    ("Nissan", "Leaf SV", 2012),
    ("Nissan", "Leaf SV", 2013),
    ("Nissan", "Leaf SV", 2014),
    ("Nissan", "Leaf SV", 2015),
    ("Nissan", "Leaf SV", 2016),
    ("Nissan", "Leaf SV", 2017),
    ("Nissan", "Leaf SV", 2018),
    ("Nissan", "Leaf SV", 2019),
    ("Nissan", "Leaf SV Plus", 2019),
    
    ("Renault", "Zoe Q210", 2013),
    ("Renault", "Zoe Q90", 2017),
    ("Renault", "Zoe Q90", 2019),
    
    ("Tesla", "Model 3 Long Range AWD", 2018),
    ("Tesla", "Model 3 Long Range AWD", 2019),
    ("Tesla", "Model 3 Long Range AWD", 2020),
    ("Tesla", "Model 3 Long Range RWD", 2017),
    ("Tesla", "Model 3 Long Range RWD", 2018),
    ("Tesla", "Model 3 Long Range RWD", 2019),
    ("Tesla", "Model 3 Mid Range RWD", 2018),
    ("Tesla", "Model 3 Mid Range RWD", 2019),
    ("Tesla", "Model 3 Performance AWD", 2018),
    ("Tesla", "Model 3 Performance AWD", 2019),
    ("Tesla", "Model 3 Performance AWD", 2020),
    ("Tesla", "Model 3 Standard Range Plus RWD", 2019),
    ("Tesla", "Model 3 Standard Range Plus RWD", 2020),
    
    ("Volkswagen", "e-Golf", 2014),
    ("Volkswagen", "e-Golf Limited Edition", 2015),
    ("Volkswagen", "e-Golf Limited Edition", 2017),
    ("Volkswagen", "e-Golf SE", 2016),
    ("Volkswagen", "e-Golf SE", 2017),
    ("Volkswagen", "e-Golf SE", 2019),
    ("Volkswagen", "e-Golf SE", 2020),
    ("Volkswagen", "e-Golf SEL Premium", 2015),
    ("Volkswagen", "e-Golf SEL Premium", 2016),
    ("Volkswagen", "e-Golf SEL Premium", 2017),
    ("Volkswagen", "ID.3", 2020),
]

station_distribution = {                  # Dictionary with charging stations type probability distribution per the purpose of the trip (location or destination)
    'prob_charging_point': {
        'errands': {'public': 0.5, 'none': 0.5},
        'escort': {'public': 0.5, 'none': 0.5},
        'leisure': {'public': 0.5, 'none': 0.5},
        'shopping': {'public': 0.5, 'none': 0.5},
        'home': {'public': 0.0,'home':1.0, 'none': 0.0}, #if at home, will always find a charger (personnal one) : assumption
        'workplace': {'public': 0.1, 'workplace': 0.9, 'none': 0.0},   # If the vehicle is at the workplace, it will always find a charging station available (assumption)
        'driving': {'none': 0.99, 'fast75': 0.005, 'fast150': 0.005}}, # with the low probability given to fast charging is to ensure fast charging only for very long trips (assumption)
    'capacity_charging_point': {                                       # Nominal power rating of charging station in kW
        'public': 22,
        'home': 3.7,
        'workplace': 11,
        'none': 0,  # dummy station
        'fast75': 75,
        'fast150': 150}
}

In [4]:
rules = ["user_defined","commuter"]

In [5]:
# Ensure output directories exist
os.makedirs("db", exist_ok=True)
os.makedirs("users_csv",exist_ok=True)
os.makedirs("final_csv",exist_ok=True)
DB = DataBase('db')

## Profiles generation

In [7]:
for i in range(10,num_users+10):
    
    # Mobility
    m = Mobility(config_folder='config_files')
    rule = random.choice(rules)
    m.set_params(
             name_prefix=f"EV_{i}",
             total_hours=simulation_hours, 
             time_step_in_hrs=time_step, # 15 minutes
             category=rule,
             reference_date="01/01/2019"
            )
    m.set_stats(
            stat_ntrip_path="TripsPerDay.csv",
            stat_dest_path="DepartureDestinationTrip.csv",
            stat_km_duration_path="DistanceDurationTrip.csv",
            )
    m.set_rules(rule_key=rule)
    m.run()
    m.save_profile(folder="db")
    
    DB.update()
    DB.loadfiles()

    # Consumption
    mname = m.name
    HI = HeatInsulation(True)
    BEVS = BEVspecs()
    EV_chosen = BEVS.model(random.choice(ev_models))
    c = Consumption(mname, EV_chosen)
    c.load_setting_mobility(DB)
    c.run(
    heat_insulation=HI,
    weather_country='DE',
    weather_year=2016,
    passenger_mass=75,                   # kg
    passenger_sensible_heat=70,          # W
    passenger_nr=1.5,                    # Passengers per vehicle including driver
    air_cabin_heat_transfer_coef=20,     # W/(m2K). Interior walls
    air_flow = 0.02,                     # m3/s. Ventilation
    driving_cycle_type='WLTC',           # Two options "WLTC" or "EPA"
    road_type=0,                         # For rolling resistance, Zero represents a new road.
    road_slope=0
    )
    c.save_profile('db')
    
    DB.update()
    DB.loadfiles()
    c.timeseries.to_csv(f"users_csv/EV_{i}.csv")


New profile running: EV_10_W53_18d11
Progress: 8% [32 / 365] days

C:\Users\Victoria Van Rillaer\AppData\Local\Programs\Python\Python310\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(
C:\Users\Victoria Van Rillaer\AppData\Local\Programs\Python\Python310\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(


Progress: 9% [34 / 365] days

C:\Users\Victoria Van Rillaer\AppData\Local\Programs\Python\Python310\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(
C:\Users\Victoria Van Rillaer\AppData\Local\Programs\Python\Python310\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(


Progress: 11% [42 / 365] days

...

Progress: 96% [351 / 365] days

C:\Users\Victoria Van Rillaer\AppData\Local\Programs\Python\Python310\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(
C:\Users\Victoria Van Rillaer\AppData\Local\Programs\Python\Python310\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(
C:\Users\Victoria Van Rillaer\AppData\Local\Programs\Python\Python310\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(


Progress: 96% [354 / 365] days

C:\Users\Victoria Van Rillaer\AppData\Local\Programs\Python\Python310\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(


Progress: 100% [365 / 365] days
Profile done: EV_109_W53_1eeae
Elapsed time (min): 0.33
 
See Log files
C:\Users\Victoria Van Rillaer\OneDrive - UCL\OneDrive - UCL\Documents\UCL\Master 2\TFE\code\datasets_generation\log\emobpy.log
Fallback value 0.9 added for missing battery_charging_eff parameter.
Fallback value 0.95 added for missing battery_discharging_eff parameter.
Fallback value 0.95 added for missing transmission_eff parameter.
Fallback value 0.3 added for missing auxiliary_power parameter.
Fallback value 3.5 added for missing cabin_volume parameter.
Fallback value 1 added for missing hvac_cop_heating parameter.
Fallback value 2 added for missing hvac_cop_cooling parameter.
New profile running: EV_109_W53_1eeae_Tesla_Model_3_Mid_Range_RWD_2018_bd318
temp Kelvin DE 2016 Timezone: Europe/Berlin
pressure Pascal DE 2016 Timezone: Europe/Berlin
dew_point Kelvin DE 2016 Timezone: Europe/Berlin
Progress: 100% [699 / 699] trips
 
See Log files
C:\Users\Victoria Van Rillaer\OneDrive - UC

## Dataset creation

In [ ]:
df = pd.read_csv('users_csv/EV_0.csv')
df.info()

In [ ]:
files = os.listdir('users_csv/')
files = [f for f in files if f.endswith('.csv')]
    
for file in files :
    match = re.search(r'(EV_\d+)\.csv', file)
    file_path1 = os.path.join('users_csv/', file)
    df1 = pd.read_csv(file_path1, parse_dates=["date"])
    
    df1 = df1[['date','state','consumption']]
    df1.rename(columns={'date': 'datetime'}, inplace=True)
    df1.to_csv(f'final_csv/{match.group(1)}.csv')
